In [0]:
'''
  fetch dataset
'''

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))
  
fileId = drive.CreateFile({'id': '1b5h1DQ3sRCjaBB5rwZ-TflO9zhHSgODw'}) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  # UMNIST.zip
fileId.GetContentFile('ass2_dataset.zip')

In [0]:
!unzip ass2_dataset.zip && mv Assignment2\ Dataset dataset

In [1]:
# necessray libraries
import tensorflow as tf
from keras.models import Sequential      # sequential way of creating NN
from keras.layers import Conv2D          # convolution layer having feature maps 
from keras.layers import MaxPooling2D    # pooling layer having pooled feature maps
from keras.layers import Flatten,Dropout # flattening cells of pooled feature map to a vec
from keras.layers import Dense,Activation   # adding fully connected layers to NN

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.constraints import max_norm
from keras.layers.normalization import BatchNormalization
from keras import optimizers

from keras.applications import VGG16
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Lambda
from keras import backend as K

import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import glob
from keras.preprocessing import image

import keras

Using TensorFlow backend.


In [0]:
# images params
width = 224
height = 224
img_shape = (width,height)
inp_shape = (height,width,3) # input size (np array dims)
b_size = 16        # batch sized

In [3]:
# path to training data
training_path = './dataset'

# get the training labels
training_labels = os.listdir(training_path)
training_labels.sort()
#training_labels = training_labels[1:] ## caution! reads .DStore too

# img
images = []
labels = []
intra_labels = []

# loop over the training data sub-folders/labels
for label in training_labels:
    
    label_directory = os.path.join(training_path, label)
    intra_labels_ = os.listdir(label_directory)
    intra_labels_.sort()
    for intra_label in intra_labels_:
        
        intra_label_directory = os.path.join(label_directory, intra_label)
        
        # iterate over each image, transforming into visual words
        for img_name in os.listdir(intra_label_directory):
            img = image.load_img(os.path.join(intra_label_directory,img_name), target_size=(224, 224))
            img = image.img_to_array(img)
            img=img/255.
            
            # collect image and its label
            images.append(img)
            labels.append(label)
            intra_labels.append(label + '_' + intra_label)


images_np = np.asarray(images)

labels_np = np.asarray(labels)
label_enc = LabelEncoder().fit_transform(labels_np).reshape(-1,1)
labels_np = OneHotEncoder().fit_transform(label_enc).toarray() 

intra_labels_np = np.asarray(intra_labels)
intra_label_enc = LabelEncoder().fit_transform(intra_labels_np).reshape(-1,1)
intra_labels_np = OneHotEncoder().fit_transform(intra_label_enc).toarray() 


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [0]:
# shuffling
images_train,_ , labels_train,_ , intra_labels_train,_ = train_test_split(images_np,
                                                                          labels_np, 
                                                                          intra_labels_np,
                                                                          test_size = 0.0,
                                                                          random_state=42)

In [5]:
images_train.shape

(2423, 224, 224, 3)

# Base Model

In [6]:
# pre trained model
base_model = keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape = inp_shape)
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_____________________________________

In [9]:
print('Base model layers:\n')
for i, layer in enumerate(base_model.layers[:]):
    print(str(i) + ' : ' + layer.name + '\n\t '+ str(layer.output.shape))

Base model layers:

0 : input_1
	 (?, 224, 224, 3)
1 : Conv1_pad
	 (?, 225, 225, 3)
2 : Conv1
	 (?, 112, 112, 32)
3 : bn_Conv1
	 (?, 112, 112, 32)
4 : Conv1_relu
	 (?, 112, 112, 32)
5 : expanded_conv_depthwise
	 (?, 112, 112, 32)
6 : expanded_conv_depthwise_BN
	 (?, 112, 112, 32)
7 : expanded_conv_depthwise_relu
	 (?, 112, 112, 32)
8 : expanded_conv_project
	 (?, 112, 112, 16)
9 : expanded_conv_project_BN
	 (?, 112, 112, 16)
10 : block_1_expand
	 (?, 112, 112, 96)
11 : block_1_expand_BN
	 (?, 112, 112, 96)
12 : block_1_expand_relu
	 (?, 112, 112, 96)
13 : block_1_pad
	 (?, 113, 113, 96)
14 : block_1_depthwise
	 (?, 56, 56, 96)
15 : block_1_depthwise_BN
	 (?, 56, 56, 96)
16 : block_1_depthwise_relu
	 (?, 56, 56, 96)
17 : block_1_project
	 (?, 56, 56, 24)
18 : block_1_project_BN
	 (?, 56, 56, 24)
19 : block_2_expand
	 (?, 56, 56, 144)
20 : block_2_expand_BN
	 (?, 56, 56, 144)
21 : block_2_expand_relu
	 (?, 56, 56, 144)
22 : block_2_depthwise
	 (?, 56, 56, 144)
23 : block_2_depthwise_BN
	

In [10]:
from keras.models import Model
base_model.layers.pop() 
model2 = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [11]:
print('Base model layers:\n')
for i, layer in enumerate(model2.layers[:]):
    print(str(i) + ' : ' + layer.name + '\n\t '+ str(layer.output.shape))

Base model layers:

0 : input_1
	 (?, 224, 224, 3)
1 : Conv1_pad
	 (?, 225, 225, 3)
2 : Conv1
	 (?, 112, 112, 32)
3 : bn_Conv1
	 (?, 112, 112, 32)
4 : Conv1_relu
	 (?, 112, 112, 32)
5 : expanded_conv_depthwise
	 (?, 112, 112, 32)
6 : expanded_conv_depthwise_BN
	 (?, 112, 112, 32)
7 : expanded_conv_depthwise_relu
	 (?, 112, 112, 32)
8 : expanded_conv_project
	 (?, 112, 112, 16)
9 : expanded_conv_project_BN
	 (?, 112, 112, 16)
10 : block_1_expand
	 (?, 112, 112, 96)
11 : block_1_expand_BN
	 (?, 112, 112, 96)
12 : block_1_expand_relu
	 (?, 112, 112, 96)
13 : block_1_pad
	 (?, 113, 113, 96)
14 : block_1_depthwise
	 (?, 56, 56, 96)
15 : block_1_depthwise_BN
	 (?, 56, 56, 96)
16 : block_1_depthwise_relu
	 (?, 56, 56, 96)
17 : block_1_project
	 (?, 56, 56, 24)
18 : block_1_project_BN
	 (?, 56, 56, 24)
19 : block_2_expand
	 (?, 56, 56, 144)
20 : block_2_expand_BN
	 (?, 56, 56, 144)
21 : block_2_expand_relu
	 (?, 56, 56, 144)
22 : block_2_depthwise
	 (?, 56, 56, 144)
23 : block_2_depthwise_BN
	

In [12]:
'''

        Fine Grained Classification::::::::


'''

'\n\n        Fine Grained Classification::::::::\n\n\n'

In [0]:
conv_base_model = model2

In [14]:
# freezing layers 
for layer in base_model.layers:
    layer.trainable = False
    
# check freezed layers
print('last 10 layers:\n')
for i, layer in enumerate(base_model.layers[-10:]):
    print(str(i) + ' : ' + layer.name + '\n\t '+ str(layer.trainable))

last 10 layers:

0 : block_15_add
	 False
1 : block_16_expand
	 False
2 : block_16_expand_BN
	 False
3 : block_16_expand_relu
	 False
4 : block_16_depthwise
	 False
5 : block_16_depthwise_BN
	 False
6 : block_16_depthwise_relu
	 False
7 : block_16_project
	 False
8 : block_16_project_BN
	 False
9 : Conv_1
	 False


# Step 1 : Learning Bilinear Layer

In [15]:
# Bilinearity by outer product of output layer of convolutional base
x = conv_base_model.output
x = Lambda(lambda x: tf.einsum('ijkm,ijkn->imn',x,x))(x)
print(x.get_shape()[1], x.get_shape()[2])
x = Lambda(lambda x: tf.reshape(x,[-1,x.get_shape()[1]* x.get_shape()[2]]))(x) #512*512
x = Lambda(lambda x: tf.multiply(tf.sign(x),tf.sqrt(tf.abs(x)+1e-12)))(x)
x = Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(x)
print(x.get_shape())

320 320
(?, 102400)


In [0]:
## fully connected network
num_classes = 36
output_layer_f = Dense(num_classes, activation= tf.nn.softmax)(x)

In [0]:
# Model
model_f = Model(inputs = base_model.input, outputs = output_layer_f)

In [18]:
model_f.summary()
model_f.compile(optimizer= optimizers.Adam(),
              loss='categorical_crossentropy',  metrics = ['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [20]:
# fit model

num_epochs = 15
model_f.fit(images_train, 
         intra_labels_train, 
         batch_size = b_size,
         epochs = num_epochs, 
         validation_split = 0.2)

Train on 1938 samples, validate on 485 samples
Epoch 1/1
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0477 - acc: 0.9995 - val_loss: 0.3319 - val_acc: 0.8990


# Step 2 : Fine tuning

In [22]:
# freezing layers 
for layer in model_f.layers:
    layer.trainable = False
    

for layer in conv_base_model.layers[-4:]:          
    layer.trainable = True


# check freezed layers
print('last 10 layers:\n')
for i, layer in enumerate(model_f.layers[:]):
    print(str(i) + ' : ' + layer.name + '\n\t '+ str(layer.trainable))

last 10 layers:

0 : input_1
	 False
1 : Conv1_pad
	 False
2 : Conv1
	 False
3 : bn_Conv1
	 False
4 : Conv1_relu
	 False
5 : expanded_conv_depthwise
	 False
6 : expanded_conv_depthwise_BN
	 False
7 : expanded_conv_depthwise_relu
	 False
8 : expanded_conv_project
	 False
9 : expanded_conv_project_BN
	 False
10 : block_1_expand
	 False
11 : block_1_expand_BN
	 False
12 : block_1_expand_relu
	 False
13 : block_1_pad
	 False
14 : block_1_depthwise
	 False
15 : block_1_depthwise_BN
	 False
16 : block_1_depthwise_relu
	 False
17 : block_1_project
	 False
18 : block_1_project_BN
	 False
19 : block_2_expand
	 False
20 : block_2_expand_BN
	 False
21 : block_2_expand_relu
	 False
22 : block_2_depthwise
	 False
23 : block_2_depthwise_BN
	 False
24 : block_2_depthwise_relu
	 False
25 : block_2_project
	 False
26 : block_2_project_BN
	 False
27 : block_2_add
	 False
28 : block_3_expand
	 False
29 : block_3_expand_BN
	 False
30 : block_3_expand_relu
	 False
31 : block_3_pad
	 False
32 : block_3_dept

In [23]:
# fit model

num_epochs = 20
model_f.fit(images_train, 
         intra_labels_train, 
         batch_size = b_size,
         epochs = num_epochs, 
         validation_split = 0.2)

Train on 1938 samples, validate on 485 samples
Epoch 1/20
  32/1938 [..............................] - ETA: 8s - loss: 0.0475 - acc: 1.0000

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1938/1938 [==============================] - 8s 4ms/step - loss: 0.0444 - acc: 0.9995 - val_loss: 0.3243 - val_acc: 0.9052
Epoch 2/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0378 - acc: 1.0000 - val_loss: 0.3179 - val_acc: 0.9052
Epoch 3/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0403 - acc: 0.9995 - val_loss: 0.3118 - val_acc: 0.9134
Epoch 4/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0359 - acc: 0.9990 - val_loss: 0.3033 - val_acc: 0.9113
Epoch 5/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0275 - acc: 0.9995 - val_loss: 0.3005 - val_acc: 0.9134
Epoch 6/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0258 - acc: 1.0000 - val_loss: 0.2958 - val_acc: 0.9175
Epoch 7/20
1938/1938 [==============================] - 8s 4ms/step - loss: 0.0286 - acc: 0.9985 - val_loss: 0.2969 - val_acc: 0.9155
Epoch 8/20
1938/1938 [==============================] - 8s 4ms/step - los

In [0]:
model_f.save('try_out_3_bcnn_fine_mobile_netv2_corrected.h5')

# Coarse Afterwards

In [126]:

'''
    Coarse :::::

'''


'\n    Coarse :::::\n\n'

In [25]:
# freezing layers 
for layer in base_model.layers:
    layer.trainable = False

# check freezed layers
print('last 10 layers:\n')
for i, layer in enumerate(base_model.layers[-10:]):
    print(str(i) + ' : ' + layer.name + '\n\t '+ str(layer.trainable))

last 10 layers:

0 : block_15_add
	 False
1 : block_16_expand
	 False
2 : block_16_expand_BN
	 False
3 : block_16_expand_relu
	 False
4 : block_16_depthwise
	 False
5 : block_16_depthwise_BN
	 False
6 : block_16_depthwise_relu
	 False
7 : block_16_project
	 False
8 : block_16_project_BN
	 False
9 : Conv_1
	 False


In [26]:
'''

    Augmenting Convolution base for coasre
    
'''
num_classes = 5
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer_c = Dense(num_classes, activation= tf.nn.softmax)(x)

# Sequential Model
model_c = Model(inputs = base_model.input, outputs = output_layer_c)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
model_c.summary()
model_c.compile(optimizer= optimizers.Adam(),
              loss='categorical_crossentropy',  metrics = ['accuracy'])



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [28]:
# fit model

num_epochs = 20
model_c.fit(images_train, 
         labels_train, 
         batch_size = b_size,
         epochs = num_epochs, 
         validation_split = 0.2)

Train on 1938 samples, validate on 485 samples
Epoch 1/20
1938/1938 [==============================] - 12s 6ms/step - loss: 5.8126 - acc: 0.6182 - val_loss: 1.8536 - val_acc: 0.8845
Epoch 2/20
1938/1938 [==============================] - 9s 5ms/step - loss: 2.2870 - acc: 0.8488 - val_loss: 1.6024 - val_acc: 0.8990
Epoch 3/20
1938/1938 [==============================] - 9s 5ms/step - loss: 1.5601 - acc: 0.8916 - val_loss: 1.5458 - val_acc: 0.9031
Epoch 4/20
1938/1938 [==============================] - 9s 5ms/step - loss: 1.2144 - acc: 0.9149 - val_loss: 0.0333 - val_acc: 0.9959
Epoch 5/20
1938/1938 [==============================] - 9s 5ms/step - loss: 0.5973 - acc: 0.9499 - val_loss: 0.3774 - val_acc: 0.9732
Epoch 6/20
1938/1938 [==============================] - 9s 5ms/step - loss: 0.5276 - acc: 0.9541 - val_loss: 0.0971 - val_acc: 0.9938
Epoch 7/20
1938/1938 [==============================] - 9s 5ms/step - loss: 0.3577 - acc: 0.9706 - val_loss: 0.0332 - val_acc: 0.9979
Epoch 8/20
193

In [0]:

model_c.save('try_out_3_bcnn_coarse_mobile_netv2_corrected.h5')